# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [97]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-ed2c25a00e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [4]:
#[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
# query example
queryString = """
SELECT DISTINCT ?s ?name ?obj ?objname
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?obj.
   # get the label 
   ?obj sc:name ?objname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('objname', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('objname', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('objname', 'basketball player')]
3


In [10]:
#[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('objname', 'basketball player')]
#[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
# query example
queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {{
   # bind something
   wd:Q3665646 ?p ?obj.
   # get the label 
   ?obj sc:name ?oname.
   }
   UNION {
   ?obj ?p wd:Q3665646.
   ?obj sc:name ?oname.
   
   }
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q18536342'), ('oname', 'competitive player')]
[('obj', 'http://www.wikidata.org/entity/P4557'), ('oname', 'Indiana Basketball Hall of Fame ID')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/P2685'), ('oname', 'Basketball Reference NBA player ID')]
[('obj', 'http://www.wikidata.org/entity/P3647'), ('oname', 'NBA.com player ID')]
[('obj', 'http://www.wikidata.org/entity/P3696'), ('oname', 'Sports-Reference.com college basketball player ID')]
[('obj', 'http://www.wikidata.org/entity/P4561'), ('oname', 'Basketball Reference WNBA player ID')]
[('obj', 'http://www.wikidata.org/entity/P4744'), ('oname', 'Basketball-Reference.com NBA G League player ID')]
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
[('obj', 'http://www.wikidata.org/entity/Q2066131'), ('oname', '

Final query for this task

In [11]:
# write your final query
#[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('objname', 'basketball player')]
#[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
# query example
queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {{
   # bind something
   wd:Q3665646 ?p ?obj.
   # get the label 
   ?obj sc:name ?oname.
   }
   UNION {
   ?obj ?p wd:Q3665646.
   ?obj sc:name ?oname.
   } 
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q18536342'), ('oname', 'competitive player')]
[('obj', 'http://www.wikidata.org/entity/P4557'), ('oname', 'Indiana Basketball Hall of Fame ID')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('oname', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/P2685'), ('oname', 'Basketball Reference NBA player ID')]
[('obj', 'http://www.wikidata.org/entity/P3647'), ('oname', 'NBA.com player ID')]
[('obj', 'http://www.wikidata.org/entity/P3696'), ('oname', 'Sports-Reference.com college basketball player ID')]
[('obj', 'http://www.wikidata.org/entity/P4561'), ('oname', 'Basketball Reference WNBA player ID')]
[('obj', 'http://www.wikidata.org/entity/P4744'), ('oname', 'Basketball-Reference.com NBA G League player ID')]
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
[('obj', 'http://www.wikidata.org/entity/Q2066131'), ('oname', '

## Task 2
Identify the BGP for basketball.

Final query for this task

In [15]:
# write your final query
#[('obj', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {{
   # bind something
   wd:Q5372 ?p ?obj.
   # get the label 
   ?obj sc:name ?oname.
   }
   UNION {
   ?obj ?p wd:Q5372.
   ?obj sc:name ?oname.
   } 
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q2341939'), ('oname', 'basketball court')]
[('obj', 'http://www.wikidata.org/entity/Q15304953'), ('oname', 'WikiProject African diaspora')]
[('obj', 'http://www.wikidata.org/entity/Q212434'), ('oname', 'Olympic sport')]
[('obj', 'http://www.wikidata.org/entity/Q216048'), ('oname', 'team sport')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q31629'), ('oname', 'type of sport')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
[('obj', 'http://www.wikidata.org/entity/Q46343203'), ('oname', "men's basketball")]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('oname', 'Wikipedia:Vital articles')]
[('obj', 'http://www.wikidata.org/entity/Q877517'), ('oname', 'ball game')]
10


## Task 3
Identify the BGP for basketball team.

Final query for this task

In [18]:
# write your final query
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {{
   # bind something
   wd:Q13393265 ?p ?obj.
   # get the label 
   ?obj sc:name ?oname.
   }
   UNION {
   ?obj ?p wd:Q13393265.
   ?obj sc:name ?oname.
   } 
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q51747567'), ('oname', 'squad')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/Q5137571'), ('oname', 'basketball coach')]
[('obj', 'http://www.wikidata.org/entity/Q847017'), ('oname', 'sports club')]
[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
[('obj', 'http://www.wikidata.org/entity/Q5648512'), ('oname', 'Template:Infobox basketball club')]
[('obj', 'http://www.wikidata.org/entity/Q5647277'), ('oname', 'Category:Basketball teams')]
[('obj', 'http://www.wikidata.org/entity/Q8206968'), ('oname', 'Hebraica y Macabi')]
[('obj', 'http://www.wikidata.org/entity/Q5144784'), ('oname', "Colgate Raiders men's basketball")]
[('obj', 'http://www.wikidata.org/entity/Q7051115'), ('oname', "Norfolk State Spartans men's basketball")]
[('obj', 'http://www.wikidata.o

## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [47]:
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
# write your final query
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
queryString = """
SELECT DISTINCT ?obj ?oname
WHERE {{
   # bind something
   wd:Q13393265 ?p ?obj.
   # get the label 
   ?obj sc:name ?oname.
   }
   UNION {
   ?obj ?p wd:Q13393265.
   ?obj sc:name ?oname.
   }
   FILTER(REGEX(?oname ,".*country*"^^xsd:string))

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
[('obj', 'http://www.wikidata.org/entity/Q7617484'), ('oname', 'Category:Basketball teams in Georgia (country)')]
2


In [48]:
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
queryString = """
SELECT DISTINCT ?obj ?objname ?oname
WHERE {
   # bind something
   wd:Q9384767 ?p ?obj.
    #?s ?p wd:Q16826020.
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?oname.

   
   #FILTER(REGEX(?oname ,"country*"^^xsd:string))

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q4167836'), ('objname', 'Wikimedia category'), ('oname', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('objname', 'basketball team'), ('oname', 'category combines topics')]
[('obj', 'http://www.wikidata.org/entity/Q19360703'), ('objname', 'by country'), ('oname', 'category combines topics')]
[('obj', 'http://www.wikidata.org/entity/Q6256'), ('objname', 'country'), ('oname', 'category combines topics')]
4


In [61]:
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?obj ?objname ?p ?oname
WHERE {
   # bind something
   wd:Q12973014 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?oname.

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/P6087'), ('objname', 'coach of sports team'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('oname', 'Wikidata property')]
[('obj', 'http://www.wikidata.org/entity/P54'), ('objname', 'member of sports team'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('oname', 'Wikidata property')]
[('obj', 'http://www.wikidata.org/entity/Q847017'), ('objname', 'sports club'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('oname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/P634'), ('objname', 'captain'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('oname', 'properties for this type')]
[('obj', 'http://www.wikidata.org/entity/P505'), ('objname', 'general manager'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('oname', 'properties for this type')]
[('obj', 'http://www.wikidata.org/entity/P831'), ('objname', 'parent club'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('oname', 'properties for

In [62]:
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?obj ?objname 
WHERE {
   # bind something
   wd:Q4438121 wdt:P279 ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
1


In [63]:
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?obj ?objname ?p ?pname
WHERE {
   # bind something
   wd:Q43229 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('obj', 'http://www.wikidata.org/entity/Q6190581'), ('objname', 'Template:Infobox organization'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('obj', 'http://www.wikidata.org/entity/Q6688962'), ('objname', 'Template:Aspects of organizations'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('obj', 'http://www.wikidata.org/entity/Q57971242'), ('objname', 'Template:Infobox organization/Wikidata'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('obj', 'http://www.wikidata.org/entity/Q3535173'), ('objname', 'organizational life cycle'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pname', 'has quality')]
[('obj', 'http://www.wikidata.org/entity/Q17197366'), ('objname', 'type of organisation'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pname', 'has quality')]
[('obj', 'http://www.wikidata.org/entity/Q730573'), ('objname', 'organizati

In [70]:
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?countryname ?pname
WHERE {
   # bind something
   ?s wdt:P31 wd:Q43229;
       wdt:P17 ?countryname.
    
   
   # get the label 
   #?s sc:name ?objname.
   ?countryname sc:name ?pname.
   

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)




Results
[('countryname', 'http://www.wikidata.org/entity/Q210036'), ('pname', 'Chechen Republic of Ichkeria')]
[('countryname', 'http://www.wikidata.org/entity/Q1065073'), ('pname', 'Song')]
[('countryname', 'http://www.wikidata.org/entity/Q244165'), ('pname', 'Republic of Artsakh')]
[('countryname', 'http://www.wikidata.org/entity/Q1008'), ('pname', 'Ivory Coast')]
[('countryname', 'http://www.wikidata.org/entity/Q1009'), ('pname', 'Cameroon')]
[('countryname', 'http://www.wikidata.org/entity/Q1019'), ('pname', 'Madagascar')]
[('countryname', 'http://www.wikidata.org/entity/Q1028'), ('pname', 'Morocco')]
[('countryname', 'http://www.wikidata.org/entity/Q1029'), ('pname', 'Mozambique')]
[('countryname', 'http://www.wikidata.org/entity/Q1030'), ('pname', 'Namibia')]
[('countryname', 'http://www.wikidata.org/entity/Q1033'), ('pname', 'Nigeria')]
[('countryname', 'http://www.wikidata.org/entity/Q1072012'), ('pname', 'international')]
[('countryname', 'http://www.wikidata.org/entity/Q117')

In [85]:
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#Copy Inception from above for date [('obj', 'http://www.wikidata.org/entity/P571'), ('objname', 'inception')
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#[('countryname', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?s ?objname ?name ?k
WHERE {
   # bind something
   ?s wdt:P31 wd:Q13393265;
       wdt:P17 ?d;
       wdt:P571 ?k.
   
   # get the label 
   ?s sc:name ?objname.
   ?d sc:name ?name.

   
   FILTER(REGEX(?name ,"Italy*"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('s', 'http://www.wikidata.org/entity/Q16586535'), ('objname', 'Pallacanestro Brindisi'), ('name', 'Italy'), ('k', '1945-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3093425'), ('objname', 'Libertas Trogylos Basket'), ('name', 'Italy'), ('k', '1970-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3515600'), ('objname', 'Taranto Cras Basket'), ('name', 'Italy'), ('k', '1961-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q36231'), ('objname', 'S.S. Felice Scandone'), ('name', 'Italy'), ('k', '1948-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q36251'), ('objname', 'Pallacanestro Biella'), ('name', 'Italy'), ('k', '1994-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q36419'), ('objname', 'Pallacanestro Reggiana'), ('name', 'Italy'), ('k', '1974-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q3967121'), ('objname', 'RB Montecatini Terme'), ('name', 'Italy'), ('k', '1949-01-01T00:00:00Z')]
[('s', 'http://www.wikid

Final query for this task

In [94]:
# write your final query
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#Copy Inception from above for date [('obj', 'http://www.wikidata.org/entity/P571'), ('objname', 'inception')
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#[('countryname', 'http://www.wikidata.org/entity/Q38'), ('pname', 'Italy')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?IRI ?TeamName (YEAR(?k) as ?Date)
WHERE {
   # bind something
   ?IRI wdt:P31 wd:Q13393265;
       wdt:P17 ?d;
       wdt:P571 ?k.
   
   # get the label 
   ?IRI sc:name ?TeamName.
   ?d sc:name ?countryname.
   

   
   FILTER(REGEX(?countryname ,"Italy*"^^xsd:string))
   FILTER( YEAR(?k) < 1945 )
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('IRI', 'http://www.wikidata.org/entity/Q3891980'), ('TeamName', 'Nuova Pallacanestro Pavia'), ('Date', '1943')]
[('IRI', 'http://www.wikidata.org/entity/Q1139737'), ('TeamName', 'Fortitudo Pallacanestro Bologna'), ('Date', '1932')]
[('IRI', 'http://www.wikidata.org/entity/Q36362'), ('TeamName', 'Olimpia Milano'), ('Date', '1936')]
[('IRI', 'http://www.wikidata.org/entity/Q36500'), ('TeamName', 'Mens Sana Basket'), ('Date', '1861')]
[('IRI', 'http://www.wikidata.org/entity/Q36264'), ('TeamName', 'Virtus Pallacanestro Bologna'), ('Date', '1927')]
[('IRI', 'http://www.wikidata.org/entity/Q36311'), ('TeamName', 'Pallacanestro Cantù'), ('Date', '1936')]
[('IRI', 'http://www.wikidata.org/entity/Q7127695'), ('TeamName', 'Pallacanestro Pavia'), ('Date', '1943')]
[('IRI', 'http://www.wikidata.org/entity/Q3754192'), ('TeamName', 'Fulgor Omegna'), ('Date', '1937')]
[('IRI', 'http://www.wikidata.org/entity/Q36572'), ('TeamName', 'Reyer Venezia Mestre'), ('Date', '1925')]
[('IRI', 'http:/

## Task 5
Identify the BGP for NBA finals

In [112]:
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]
#('obj', 'http://www.wikidata.org/entity/Q847017'), ('oname', 'sports club')]



queryString = """
SELECT DISTINCT ?obj ?objname ?p ?pname
WHERE {{
   # bind something
   wd:Q847017 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"NBA*"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
[('obj', 'http://www.wikidata.org/entity/Q2657718'), ('objname', 'Armenian Soviet Encyclopedia'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q19467997'), ('objname', 'Template:Infobox athletics club'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('obj', 'http://www.wikidata.org/entity/Q10730006'), ('objname', 'Template:Infobox sports team'), ('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('obj', 'http://www.wikidata.org/entity/P4999'), ('objname', 'number of clubs'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pname', 'Wikidata property')]
[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('objname', 'sports team'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('obj', 'http://www.wikidata.org/entity/Q51747567'), ('objname', 'squad'), ('p', 'http://www.wikidata

In [116]:
# write your queries
#[('obj', 'http://www.wikidata.org/entity/Q43229'), ('objname', 'organization')]
#[('obj', 'http://www.wikidata.org/entity/Q4438121'), ('objname', 'sports organization'), 
#('p', 'http://www.wikidata.org/prop/direct/P279'), ('oname', 'subclass of')]
#[('obj', 'http://www.wikidata.org/entity/Q8797519'), ('oname', 'Category:Sports teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q9384767'), ('oname', 'Category:Basketball teams by country')]
#[('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oname', 'sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q16826020'), ('oname', '1954 Milan High School basketball team')]
#Nationality -> wdt:P27
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]
#('obj', 'http://www.wikidata.org/entity/Q847017'), ('oname', 'sports club')]
#[('obj', 'http://www.wikidata.org/entity/P4999'), ('objname', 'number of clubs')



queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
   ?obj wdt:P4999 wd:Q4438121.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"NBA*"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)




Results
Empty


In [120]:
#[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
   ?obj ?p wd:Q3665646.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"james"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8192401'), ('objname', 'Aitor Zárate')]
[('obj', 'http://www.wikidata.org/entity/Q13708767'), ('objname', 'Aristide Landi')]
[('obj', 'http://www.wikidata.org/entity/Q15477486'), ('objname', 'Małgorzata Janowicz')]
[('obj', 'http://www.wikidata.org/entity/Q15737093'), ('objname', 'Jarrett Durham')]
[('obj', 'http://www.wikidata.org/entity/Q15843419'), ('objname', 'Ryan Brooks')]
[('obj', 'http://www.wikidata.org/entity/Q15850750'), ('objname', 'Thomas Hering')]
[('obj', 'http://www.wikidata.org/entity/Q15856626'), ('objname', 'Ama Degbeon')]
[('obj', 'http://www.wikidata.org/entity/Q15870814'), ('objname', 'Josef Klíma')]
[('obj', 'http://www.wikidata.org/entity/Q15879801'), ('objname', 'Niels Vorenhout')]
[('obj', 'http://www.wikidata.org/entity/Q15881370'), ('objname', 'Roel Aarts')]
[('obj', 'http://www.wikidata.org/entity/Q15889301'), ('objname', 'Charlie Westbrook')]
[('obj', 'http://www.wikidata.org/entity/Q15908865'), ('objname',

In [124]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
   wd:Q55216167 ?obj ?p.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"james"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P106'), ('objname', 'occupation')]
[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
[('obj', 'http://www.wikidata.org/prop/direct/P1532'), ('objname', 'country for sport')]
[('obj', 'http://www.wikidata.org/prop/direct/P1889'), ('objname', 'different from')]
[('obj', 'http://www.wikidata.org/prop/direct/P19'), ('objname', 'place of birth')]
[('obj', 'http://www.wikidata.org/prop/direct/P2067'), ('objname', 'mass')]
[('obj', 'http://www.wikidata.org/prop/direct/P2094'), ('objname', 'competition class')]
[('obj', 'http://www.wikidata.org/prop/direct/P21'), ('objname', 'sex or gender')]
[('obj', 'http://www.wikidata.org/prop/direct/P2671'), ('objname', 'Google Knowledge Graph ID')]
[('obj', 'http://www.wikidata.org/prop/direct/P27'), ('objname', 'country of citizenship')]
[('obj', 'http://www.wikidata.org/prop/direct/P31'), ('objname', 'instance of')]
[('obj', 'http://www.wikidata.org/prop/direct/P3527'), ('

In [128]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
#[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
   ?p wdt:P118 ?obj .
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   FILTER(REGEX(?objname ,"NBA"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
[('obj', 'http://www.wikidata.org/entity/Q351478'), ('objname', 'NBA')]
2


In [135]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
#[('obj', 'http://www.wikidata.org/entity/Q351478'), ('objname', 'NBA')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
    wd:Q842793 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"james"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q125'), ('objname', 'November')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q46343203'), ('objname', "men's basketball")]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('objname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
[('obj', 'http://www.wikidata.org/entity/P4744'), ('objname', 'Basketball-Reference.com NBA G League player ID')]
[('obj', 'http://www.wikidata.org/entity/Q623109'), ('objname', 'sports league')]
[('obj', 'http://www.wikidata.org/entity/Q1933075'), ('objname', 'Rio Grande Valley Vipers')]
[('obj', 'http://www.wikidata.org/entity/Q30079437'), ('objname', '2017–18 NBA Gatorade League season')]
[('obj', 'http://www.wikidata.org/entity/Q7485512'), ('objname', 'Category:NBA G League')]
10


In [139]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
     ?p ?obj wd:Q155223.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?pname ,"james"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/prop/direct/P1591'), ('objname', 'defendant')]
[('obj', 'http://www.wikidata.org/prop/direct/P1423'), ('objname', 'template has topic')]
[('obj', 'http://www.wikidata.org/prop/direct/P101'), ('objname', 'field of work')]
[('obj', 'http://www.wikidata.org/prop/direct/P108'), ('objname', 'employer')]
[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
[('obj', 'http://www.wikidata.org/prop/direct/P1269'), ('objname', 'facet of')]
[('obj', 'http://www.wikidata.org/prop/direct/P127'), ('objname', 'owned by')]
[('obj', 'http://www.wikidata.org/prop/direct/P1343'), ('objname', 'described by source')]
[('obj', 'http://www.wikidata.org/prop/direct/P1366'), ('objname', 'replaced by')]
[('obj', 'http://www.wikidata.org/prop/direct/P137'), ('objname', 'operator')]
[('obj', 'http://www.wikidata.org/prop/direct/P264'), ('objname', 'record label')]
[('obj', 'http://www.wikidata.org/prop/direct/P301'), ('objname', "category's main top

In [141]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
     ?obj wdt:P1346 ?p.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   FILTER(REGEX(?objname ,"NBA"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q600389'), ('objname', '1990–91 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q2264531'), ('objname', '1967–68 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q241834'), ('objname', '1989–90 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q602553'), ('objname', '1969–70 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q2294127'), ('objname', '1971–72 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q113619'), ('objname', '2012–13 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q16191100'), ('objname', '2014-15 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q19862716'), ('objname', '2015–16 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q918054'), ('objname', 'NBA Coach of the Year Award')]
[('obj', 'http://www.wikidata.org/entity/Q631240'), ('objname', '1979–80 NBA season')]
[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
[('obj', 'http://www.wikid

In [174]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
#[('obj', 'http://www.wikidata.org/entity/Q4600307'), ('objname', '2002 NBA Finals')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
     wd:Q4600307 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }

   
   #FILTER(REGEX(?objname ,"NBA"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('objname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/Q842375'), ('objname', 'NBA Finals')]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('objname', 'Los Angeles Lakers')]
3


In [144]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
#[('obj', 'http://www.wikidata.org/entity/Q4600307'), ('objname', '2002 NBA Finals')]
#[('obj', 'http://www.wikidata.org/entity/Q842375'), ('objname', 'NBA Finals')]
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
     wd:Q842375 ?p ?obj.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1366722'), ('objname', 'final')]
[('obj', 'http://www.wikidata.org/entity/Q118'), ('objname', 'April')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('objname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/Q15275719'), ('objname', 'recurring event')]
[('obj', 'http://www.wikidata.org/entity/Q14335327'), ('objname', 'Template:Infobox basketball final')]
[('obj', 'http://www.wikidata.org/entity/Q8663431'), ('objname', 'Category:National Basketball Association Finals')]
7


Final query for this task

In [145]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {{
   # bind something
     wd:Q842375 ?p ?obj.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1366722'), ('objname', 'final')]
[('obj', 'http://www.wikidata.org/entity/Q118'), ('objname', 'April')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('objname', 'basketball')]
[('obj', 'http://www.wikidata.org/entity/Q15275719'), ('objname', 'recurring event')]
[('obj', 'http://www.wikidata.org/entity/Q14335327'), ('objname', 'Template:Infobox basketball final')]
[('obj', 'http://www.wikidata.org/entity/Q8663431'), ('objname', 'Category:National Basketball Association Finals')]
7


## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [150]:
# write your queries
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
##[('obj', 'http://www.wikidata.org/entity/Q842375'), ('objname', 'NBA Finals')]
#wd:Q36159	Lebron James	node
#wd:Q29545	Kevin Durant	node

queryString = """
SELECT DISTINCT ?obj ?objname ?s ?pname
WHERE {
   
   # bind something
    ?s wdt:P54 ?obj.
   # get the label 
   ?obj sc:name ?objname.
   ?s sc:name ?pname.
   VALUES ?s {wd:Q36159 wd:Q29545}
   
  
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q180950'), ('objname', 'Oklahoma City Thunder'), ('s', 'http://www.wikidata.org/entity/Q29545'), ('pname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q235326'), ('objname', 'Seattle SuperSonics'), ('s', 'http://www.wikidata.org/entity/Q29545'), ('pname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q572134'), ('objname', 'Brooklyn Nets'), ('s', 'http://www.wikidata.org/entity/Q29545'), ('pname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('objname', 'Miami Heat'), ('s', 'http://www.wikidata.org/entity/Q36159'), ('pname', 'LeBron James')]
[('obj', 'http://www.wikidata.org/entity/Q7707830'), ('objname', "Texas Longhorns men's basketball"), ('s', 'http://www.wikidata.org/entity/Q29545'), ('pname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q244847'), ('objname', "United States men's national basketball team"), ('s', 'http://www.wikidata.org/entity/Q29545'), ('pname', 'Kevi

In [154]:
# write your queries
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
##[('obj', 'http://www.wikidata.org/entity/Q842375'), ('objname', 'NBA Finals')]
#wd:Q36159	Lebron James	node
#wd:Q29545	Kevin Durant	node
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
#[('obj', 'http://www.wikidata.org/entity/Q4600307'), ('objname', '2002 NBA Finals')]

queryString = """
SELECT DISTINCT ?obj ?objname ?pname
WHERE {
   
   # bind something
    wd:Q4600307 ?p ?obj.
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
   #VALUES ?s {wd:Q36159 wd:Q29545}
   
  
}
LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('objname', 'Los Angeles Lakers'), ('pname', 'winner')]
[('obj', 'http://www.wikidata.org/entity/Q842375'), ('objname', 'NBA Finals'), ('pname', 'instance of')]
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('objname', 'basketball'), ('pname', 'sport')]
3


In [161]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?objname ?p ?pname
WHERE {{
   # bind something
     ?obj wdt:P1346 ?p.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
   }
   FILTER(REGEX(?objname ,"NBA Final"^^xsd:string))
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4598928'), ('objname', '2001 NBA Finals'), ('p', 'http://www.wikidata.org/entity/Q121783'), ('pname', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q4588662'), ('objname', '1993 NBA Finals'), ('p', 'http://www.wikidata.org/entity/Q128109'), ('pname', 'Chicago Bulls')]
[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award'), ('p', 'http://www.wikidata.org/entity/Q555100'), ('pname', 'Cedric Maxwell')]
[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award'), ('p', 'http://www.wikidata.org/entity/Q352162'), ('pname', 'Dennis Johnson')]
[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award'), ('p', 'http://www.wikidata.org/entity/Q179051'), ('pname', 'Kareem Abdul-Jabbar')]
[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bi

In [181]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?objname ?team_name ?Fin ?Fname
WHERE {
   # bind something
    ?s wdt:P54 ?k.
    ?obj wdt:P1346 ?k.
    ?obj wdt:P31 ?Fin.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   ?k sc:name ?team_name.
   ?Fin sc:name ?Fname.

   
   FILTER(REGEX(?objname ,"NBA Final"^^xsd:string))
   VALUES ?s {wd:Q36159 wd:Q29545}
   
   

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13405328'), ('objname', '2013 NBA Finals'), ('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q1366722'), ('Fname', 'final')]
[('obj', 'http://www.wikidata.org/entity/Q2628013'), ('objname', '2012 NBA Finals'), ('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals')]
[('obj', 'http://www.wikidata.org/entity/Q4584557'), ('objname', '1988 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals')]
[('obj', 'http://www.wikidata.org/entity/Q644353'), ('objname', '2010 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals')]
[('obj', 'http://www.wikidata.org/entity/Q170590'), ('objname', '2006 NBA Finals'), ('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals')]
[('obj', 'http://www.wikidata.o

In [185]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT ?obj ?objnam ?team_name ?Fin ?Fname (COUNT (?k) as ?winners)
WHERE {
   # bind something
    ?s wdt:P54 ?k.
    ?obj wdt:P1346 ?k.
    ?obj wdt:P31 ?Fin.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   ?k sc:name ?team_name.
   ?Fin sc:name ?Fname.

   
   FILTER(REGEX(?objname ,"NBA Final"^^xsd:string))
   VALUES ?s {wd:Q36159 wd:Q29545}

}
Group by ?objname ?team_name ?Fname ?Fin ?obj
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2628013'), ('objname', '2012 NBA Finals'), ('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
[('obj', 'http://www.wikidata.org/entity/Q4600307'), ('objname', '2002 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
[('obj', 'http://www.wikidata.org/entity/Q4584557'), ('objname', '1988 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
[('obj', 'http://www.wikidata.org/entity/Q4598928'), ('objname', '2001 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
[('obj', 'http://www.wikidata.org/entity/Q644353'), ('objname', '2010 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('Fin', 'http:/

In [187]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT  ?team_name ?Fin ?Fname (COUNT (?k) as ?winners)
WHERE {
   # bind something
    ?s wdt:P54 ?k.
    ?obj wdt:P1346 ?k.
    ?obj wdt:P31 ?Fin.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   ?k sc:name ?team_name.
   ?Fin sc:name ?Fname.

   
   FILTER(REGEX(?Fname ,"NBA Final"^^xsd:string))
   VALUES ?s {wd:Q36159 wd:Q29545}

}
Group by  ?team_name ?Fname ?Fin 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '16')]
[('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '2')]
[('team_name', 'Golden State Warriors'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '4')]
[('team_name', 'Seattle SuperSonics'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
4


In [171]:
#[('obj', 'http://www.wikidata.org/entity/Q55216167'), ('objname', 'Steven Cook')]
##[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
queryString = """
SELECT DISTINCT ?objname ?team_name (COUNT (?k) as ?winners)
WHERE {
   # bind something

    ?s wdt:P54 ?k.
    
    OPTIONAL{
    ?obj wdt:P1346 ?k.
    }
   
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   ?k sc:name ?team_name.

   
   FILTER(REGEX(?objname ,"NBA Final"^^xsd:string))
   VALUES ?s {wd:Q36159 wd:Q29545}

}
Group by ?team_name ?objname
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('objname', '2000 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
[('objname', '2001 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
[('objname', '2006 NBA Finals'), ('team_name', 'Miami Heat'), ('winners', '1')]
[('objname', '2015 NBA Finals'), ('team_name', 'Golden State Warriors'), ('winners', '1')]
[('objname', '2012 NBA Finals'), ('team_name', 'Miami Heat'), ('winners', '1')]
[('objname', '2002 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
[('objname', '1988 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
[('objname', '2009 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
[('objname', '2013 NBA Finals'), ('team_name', 'Miami Heat'), ('winners', '1')]
[('objname', '2010 NBA Finals'), ('team_name', 'Los Angeles Lakers'), ('winners', '1')]
10


Final query for this task

In [188]:
# write your final query
queryString = """
SELECT DISTINCT  ?team_name ?Fin ?Fname (COUNT (?k) as ?winners)
WHERE {
   # bind something
    ?s wdt:P54 ?k.
    ?obj wdt:P1346 ?k.
    ?obj wdt:P31 ?Fin.
   # get the label 
   ?obj sc:name ?objname.
   #?p sc:name ?pname.
   ?k sc:name ?team_name.
   ?Fin sc:name ?Fname.

   
   FILTER(REGEX(?Fname ,"NBA Final"^^xsd:string))
   VALUES ?s {wd:Q36159 wd:Q29545}

}
Group by  ?team_name ?Fname ?Fin 
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team_name', 'Los Angeles Lakers'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '16')]
[('team_name', 'Miami Heat'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '2')]
[('team_name', 'Golden State Warriors'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '4')]
[('team_name', 'Seattle SuperSonics'), ('Fin', 'http://www.wikidata.org/entity/Q842375'), ('Fname', 'NBA Finals'), ('winners', '1')]
4


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [201]:
# write your queries
#[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]

queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   
    ?obj wdt:P31 wd:Q13393265;
        wdt:P118 wd:Q155223.
   # get the label 
   ?obj sc:name ?objname.

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q131364'), ('objname', 'New York Knicks')]
[('obj', 'http://www.wikidata.org/entity/Q162954'), ('objname', 'Denver Nuggets')]
[('obj', 'http://www.wikidata.org/entity/Q167253'), ('objname', 'Portland Trail Blazers')]
[('obj', 'http://www.wikidata.org/entity/Q180950'), ('objname', 'Oklahoma City Thunder')]
[('obj', 'http://www.wikidata.org/entity/Q976396'), ('objname', 'Los Angeles Clippers')]
[('obj', 'http://www.wikidata.org/entity/Q166105'), ('objname', 'Sacramento Kings')]
[('obj', 'http://www.wikidata.org/entity/Q170649'), ('objname', 'Utah Jazz')]
[('obj', 'http://www.wikidata.org/entity/Q138089'), ('objname', 'Philadelphia 76ers')]
[('obj', 'http://www.wikidata.org/entity/Q161345'), ('objname', 'Houston Rockets')]
[('obj', 'http://www.wikidata.org/entity/Q164177'), ('objname', 'Phoenix Suns')]
[('obj', 'http://www.wikidata.org/entity/Q172339'), ('objname', 'New Orleans Pelicans')]
[('obj', 'http://www.wikidata.org/entity/Q161337'),

In [204]:
# write your queries
#[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#('s', 'http://www.wikidata.org/prop/direct/P17'), ('oname', 'country')]

queryString = """
SELECT DISTINCT ?obj ?objname ?p ?pname
WHERE {
   # bind something
   
    wd:Q202311 ?p ?obj.
        
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2277'), ('objname', 'Roman Empire'), ('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('obj', 'http://www.wikidata.org/entity/Q2320005'), ('objname', 'Ptolemaic Kingdom'), ('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces')]
[('obj', 'http://www.wikidata.org/entity/Q20987778'), ('objname', 'Sasanian Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('obj', 'http://www.wikidata.org/entity/Q1747689'), ('objname', 'Ancient Rome'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('obj', 'http://www.wikidata.org/entity/Q2277'), ('objname', 'Roman Empire'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pname', 'time period')]
[('obj', 'http://www.wikidata.org/entity/Q15'), ('objname', 'Africa'), ('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
[('obj', 'http://w

In [209]:
#('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]

# write your queries
#[('obj', 'http://www.wikidata.org/prop/direct/P118'), ('objname', 'league')]
#[('obj', 'http://www.wikidata.org/entity/Q842793'), ('objname', 'NBA G League')]
#[('obj', 'http://www.wikidata.org/entity/Q155223'), ('objname', 'National Basketball Association')]
#[('obj', 'http://www.wikidata.org/prop/direct/P54'), ('objname', 'member of sports team')]
#[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('oname', 'basketball team')]
#[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
# wdt:P106	profession	predicate

queryString = """
SELECT DISTINCT ?Cont ?Cont_name (COUNT (DISTINCT ?pl) as ?Players)
WHERE {
   # bind something
   
    ?obj wdt:P31 wd:Q13393265;
        wdt:P118 wd:Q155223.
        
    ?pl wdt:P31 wd:Q5;
        wdt:P106 wd:Q3665646;
        wdt:P54 ?obj;
        wdt:P27 ?c.
    
    ?c wdt:P30 ?Cont.
    
    ?Cont sc:name ?Cont_name

}
Group by (?Cont) (?Cont_name)

LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('Cont', 'http://www.wikidata.org/entity/Q46'), ('Cont_name', 'Europe'), ('Players', '300')]
[('Cont', 'http://www.wikidata.org/entity/Q538'), ('Cont_name', 'Insular Oceania'), ('Players', '3770')]
[('Cont', 'http://www.wikidata.org/entity/Q15'), ('Cont_name', 'Africa'), ('Players', '60')]
[('Cont', 'http://www.wikidata.org/entity/Q48'), ('Cont_name', 'Asia'), ('Players', '73')]
[('Cont', 'http://www.wikidata.org/entity/Q5401'), ('Cont_name', 'Eurasia'), ('Players', '17')]
[('Cont', 'http://www.wikidata.org/entity/Q18'), ('Cont_name', 'South America'), ('Players', '37')]
[('Cont', 'http://www.wikidata.org/entity/Q49'), ('Cont_name', 'North America'), ('Players', '3820')]
7


Final query for this task

In [210]:
# write your final query
queryString = """
SELECT DISTINCT ?Cont ?Cont_name (COUNT (DISTINCT ?pl) as ?Players)
WHERE {
   # bind something
   
    ?obj wdt:P31 wd:Q13393265;
        wdt:P118 wd:Q155223.
        
    ?pl wdt:P31 wd:Q5;
        wdt:P106 wd:Q3665646;
        wdt:P54 ?obj;
        wdt:P27 ?c.
    
    ?c wdt:P30 ?Cont.
    
    ?Cont sc:name ?Cont_name

}
Group by (?Cont) (?Cont_name)

LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('Cont', 'http://www.wikidata.org/entity/Q46'), ('Cont_name', 'Europe'), ('Players', '300')]
[('Cont', 'http://www.wikidata.org/entity/Q538'), ('Cont_name', 'Insular Oceania'), ('Players', '3770')]
[('Cont', 'http://www.wikidata.org/entity/Q15'), ('Cont_name', 'Africa'), ('Players', '60')]
[('Cont', 'http://www.wikidata.org/entity/Q48'), ('Cont_name', 'Asia'), ('Players', '73')]
[('Cont', 'http://www.wikidata.org/entity/Q5401'), ('Cont_name', 'Eurasia'), ('Players', '17')]
[('Cont', 'http://www.wikidata.org/entity/Q18'), ('Cont_name', 'South America'), ('Players', '37')]
[('Cont', 'http://www.wikidata.org/entity/Q49'), ('Cont_name', 'North America'), ('Players', '3820')]
7


## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

In [190]:
# write your queries

#[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award')

queryString = """
SELECT DISTINCT ?s ?sname ?p ?pname
WHERE {
   # bind something
   ?s ?p wd:Q739499.
   
   # get the label 
   ?s sc:name ?sname.
   ?p sc:name ?pname.
  

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q273256'), ('sname', 'Hakeem Olajuwon'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q298512'), ('sname', 'Isiah Thomas'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q362872'), ('sname', 'James Worthy'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q182455'), ('sname', 'Wilt Chamberlain'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q298531'), ('sname', 'Chauncey Billups'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q380515'), ('sname', 'Willis Reed'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q

In [191]:
#('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
# write your queries

#[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award')

queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q29545 wdt:P166 ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   
  

}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q7865896'), ('objname', 'USBWA National Freshman of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q1418009'), ('objname', 'NBA All-Rookie Team')]
[('obj', 'http://www.wikidata.org/entity/Q1641970'), ('objname', 'John R. Wooden Award')]
[('obj', 'http://www.wikidata.org/entity/Q222047'), ('objname', 'NBA Most Valuable Player Award')]
[('obj', 'http://www.wikidata.org/entity/Q31391'), ('objname', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('obj', 'http://www.wikidata.org/entity/Q4835465'), ('objname', 'BET Award for Athlete of the Century')]
[('obj', 'http://www.wikidata.org/entity/Q4896724'), ('objname', 'Best Male Athlete ESPY Award')]
[('obj', 'http://www.wikidata.org/entity/Q644357'), ('objname', 'NBA Rookie of the Year Award')]
[('obj', 'http://www.wikidata.org/entity/Q674359'), ('objname', 'All-NBA Team')]
[('obj', 'http://www.wikidata.org/entity/Q943106'), ('objname', 'Best NBA Player ESPY Award')]
[('obj',

In [194]:
#('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
# write your queries

#[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award')
#[('obj', 'http://www.wikidata.org/entity/Q222047'), ('objname', 'NBA Most Valuable Player Award')]

queryString = """
SELECT DISTINCT ?obj ?objname 
WHERE {
   # bind something
   wd:Q222047 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('obj', 'http://www.wikidata.org/entity/Q29545'), ('objname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q205326'), ('objname', 'Derrick Rose')]
[('obj', 'http://www.wikidata.org/entity/Q192707'), ('objname', 'Charles Barkley')]
[('obj', 'http://www.wikidata.org/entity/Q192955'), ('objname', 'Steve Nash')]
[('obj', 'http://www.wikidata.org/entity/Q182455'), ('objname', 'Wilt Chamberlain')]
[('obj', 'http://www.wikidata.org/entity/Q380515'), ('objname', 'Willis Reed')]
[('obj', 'http://www.wikidata.org/entity/Q8991894'), ('objname', 'Giannis Antetokounmpo')]
[('obj', 'http://www.wikidata.org/entity/Q209921'), ('objname', 'Julius Erving')]
[('obj', 'http://www.wikidata.org/entity/Q134183'), ('objname', 'Magic Johnson')]
[('obj', 'http://www.wikidata.org/entity/Q273256'), ('objname', 'Hakeem Olajuwon')]
[('obj', 'http://www.wikidata.org/entity/Q190094'), ('objname', 'Allen Iverson')]
[('obj', 'http://www.wikidata.org/entity/Q316179'), ('objname', 'Moses Malone')]
[

Final query for this task

In [195]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?objname 
WHERE {
   # bind something
   wd:Q222047 ?p ?obj.
   
   # get the label 
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q29545'), ('objname', 'Kevin Durant')]
[('obj', 'http://www.wikidata.org/entity/Q205326'), ('objname', 'Derrick Rose')]
[('obj', 'http://www.wikidata.org/entity/Q192707'), ('objname', 'Charles Barkley')]
[('obj', 'http://www.wikidata.org/entity/Q192955'), ('objname', 'Steve Nash')]
[('obj', 'http://www.wikidata.org/entity/Q182455'), ('objname', 'Wilt Chamberlain')]
[('obj', 'http://www.wikidata.org/entity/Q380515'), ('objname', 'Willis Reed')]
[('obj', 'http://www.wikidata.org/entity/Q8991894'), ('objname', 'Giannis Antetokounmpo')]
[('obj', 'http://www.wikidata.org/entity/Q209921'), ('objname', 'Julius Erving')]
[('obj', 'http://www.wikidata.org/entity/Q134183'), ('objname', 'Magic Johnson')]
[('obj', 'http://www.wikidata.org/entity/Q273256'), ('objname', 'Hakeem Olajuwon')]
[('obj', 'http://www.wikidata.org/entity/Q190094'), ('objname', 'Allen Iverson')]
[('obj', 'http://www.wikidata.org/entity/Q316179'), ('objname', 'Moses Malone')]
[

## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [200]:
# write your queries
#('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
# write your queries

#[('obj', 'http://www.wikidata.org/entity/Q739499'), ('objname', 'Bill Russell NBA Finals Most Valuable Player Award')
#[('obj', 'http://www.wikidata.org/entity/Q222047'), ('objname', 'NBA Most Valuable Player Award')]
#[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oname', 'basketball player')]
#[('obj', 'http://www.wikidata.org/prop/direct/P1346'), ('objname', 'winner')]
# wdt:P106	profession	predicate


queryString = """
SELECT DISTINCT ?p ?pname COUNT((wd:Q222047) as ?awards)
WHERE {
   # bind something
   ?p wdt:P106 wd:Q3665646;
   wdt:P166 wd:Q222047.   
   # get the label 
   ?p sc:name ?pname.
   #VALUES ?k {}
}
Group by ?p ?pname

LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/entity/Q264393'), ('pname', 'Dave Cowens'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q380515'), ('pname', 'Willis Reed'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q192955'), ('pname', 'Steve Nash'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q136678'), ('pname', 'James Harden'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q179051'), ('pname', 'Kareem Abdul-Jabbar'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q25369'), ('pname', 'Kobe Bryant'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q36159'), ('pname', 'LeBron James'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q41421'), ('pname', 'Michael Jordan'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q196185'), ('pname', 'Russell Westbrook'), ('callret-2', '1')]
[('p', 'http://www.wikidata.org/entity/Q205326'), ('pname', 'Derrick Rose'), ('callret-2', '1')]
[('p', 'http://w

Final query for this task

In [ ]:
# write your final query